In [69]:
from openalea.strawberry.import_mtgfile import import_mtgfile

import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

from openalea.strawberry.analysis import (extract_at_module_scale, occurence_module_order_along_time, df2waffle, plot_waffle,)
from openalea.strawberry.application.app_module_scale import (plot_module_pointwisemean, plot_module_occurence_module_order_along_time, plot_module_distribution_complete_incomplete_module_order,
                                          plot_module_distribution_complete_incomplete_date, plot_module_crown)
from openalea.strawberry.application.misc import get_vid_of_genotype
from openalea.strawberry.analysis import (extract_at_node_scale,prob_axillary_production)

In [54]:
list_genotypes = ["Capriss", "Ciflorette", "Cir107", "Clery", "Darselect", "Gariguette"]
all_mtg = import_mtgfile(filename=list_genotypes)


In [55]:
colormap_used = plt.cm.inferno
nb_genotypes = len(list_genotypes)

def get_colormap(cm_used, nb_color):

    values = range(0,nb_color)
    color_map = {val: colormap_used(i/len(values)) for i, val in enumerate(values)}
    for i in values:
        color_map[i] = color_map[i][:-1]
        color_map[i] = [int(255*x) for x in color_map[i]]
        color_map[i] = 'rgb'+str(color_map[i])
        s = list(color_map[i])
        s[3]='('
        s[-1]=')'
        color_map[i] = "".join(s)
    return color_map
    
color_map = get_colormap(colormap_used, nb_genotypes)

In [42]:
color_map




{0: 'rgb(0, 0, 3)',
 1: 'rgb(50, 9, 93)',
 2: 'rgb(120, 28, 109)',
 3: 'rgb(187, 55, 84)',
 4: 'rgb(237, 104, 37)',
 5: 'rgb(251, 181, 26)'}

In [16]:
df=extract_at_module_scale(all_mtg)
df

,Genotype,date,modality,plant,order,nb_visible_leaves,nb_foliar_primordia,nb_total_leaves,nb_open_flowers,nb_aborted_flowers,...,nb_vegetative_bud,nb_initiated_bud,nb_floral_bud,nb_stolons,type_of_crown,crown_status,complete_module,stage,vid,plant_vid
0,Clery,2014/12/10,A,1,0,11,2,13,0,0,...,1,0,8,3,1,3,False,H,2,1
1,Clery,2014/12/10,A,2,0,11,3,14,0,0,...,5,2,4,3,1,3,False,H,102,101
2,Clery,2014/12/10,A,3,0,8,2,10,0,0,...,3,0,5,2,1,3,False,H,189,188
3,Clery,2014/12/10,A,4,0,4,0,4,0,0,...,0,0,0,3,1,4,True,cut,255,254
4,Clery,2014/12/10,A,4,1,2,2,4,0,0,...,0,0,4,0,2,3,False,H,266,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,Gariguette,2015/05/19,A,9,2,3,0,3,2,0,...,2,0,0,1,3,4,True,65,37759,37634
1792,Gariguette,2015/05/19,A,9,2,6,0,6,10,0,...,3,0,1,0,2,4,True,87,37780,37634
1793,Gariguette,2015/05/19,A,9,3,3,0,3,1,0,...,1,0,0,1,3,4,True,65,37806,37634
1794,Gariguette,2015/05/19,A,9,3,3,0,3,5,0,...,1,0,0,1,2,4,True,77,37822,37634


### Plot pointwise 

In [43]:
layout={"xlabel": "Order",
        "ylabel": "Mean no. leaves",
        "title": "A. Leaves"}


In [44]:
fig = plot_module_pointwisemean(df, "nb_visible_leaves", layout=layout)

for i in range(0, nb_genotypes):
    fig['data'][i]['line']['color']=color_map[i]
    
layout=go.Layout(plot_bgcolor='rgba(0,0,0,0)',
                 xaxis={'visible':True, 'color':'black', 
                        'showline':True, 'linecolor': 'black',
#                         'showgrid':True, 'gridcolor':'black',
                        'tickmode':'array',
                       },
                 yaxis={'showline':True, 'linecolor': 'black',
                        'showgrid':True, 'gridcolor':'lightgrey'
                       },
                 

                )
fig.update_layout(layout)

FigureWidget({
    'data': [{'error_y': {'array': [1.9806894450790757, 1.1125596239016966,
                   …

### Plot auxiliary production

In [79]:
# SELECT THE GENOTYPE by changing the list number (0=Capriss)
genotype = list_genotypes[0]


vids=get_vid_of_genotype(all_mtg, genotypes=genotype)

df = prob_axillary_production(all_mtg,order=0,vids=vids)


In [88]:
fig = df.iplot(kind = "line",
               mode = 'lines+markers',
               xTitle="Node rank",
               yTitle='Probability',
               title=genotype,
               asFigure=True)


for i in range(0, nb_genotypes):
    fig['data'][i]['line']['color']=color_map[i]
    
layout=go.Layout(plot_bgcolor='rgba(0,0,0,0)',
                 xaxis={'visible':True, 'color':'black', 
                        'showline':True, 'linecolor': 'black',
#                         'showgrid':True, 
                        'gridcolor':'white',
                        'tick0': 1,
                        'dtick':1,
                        'tickmode':'linear',
                       },
                 yaxis={'showline':True, 'linecolor': 'black',
                        'showgrid':True, 'gridcolor':'lightgrey'
                       },
                 

                )
fig.update_layout(layout)